# 分布式回归模型

### 1. 准备工作

配置和启动 PySpark：

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

利用 Numpy 生成模拟数据，并写入文件。首先生成 $n\gg p$ 的数据：

In [ ]:
import numpy as np
np.set_printoptions(linewidth=100)

np.random.seed(123)
n = 100000
p = 100
x1 = np.random.normal(size=(n, p))
beta1 = np.random.normal(size=p)
y1 = x1.dot(beta1) + np.random.normal(scale=0.1, size=n)
dat = np.hstack((y1.reshape(n, 1), x1))
np.savetxt("data/reg_tall.txt", dat, fmt="%f", delimiter="\t")

以及 $n<p$ 的数据：

In [ ]:
n = 500
p = 5000
x2 = np.random.normal(size=(n, p))
beta2 = np.random.normal(size=p)
beta2[10:] = 0.0
y2 = x2.dot(beta2) + np.random.normal(scale=0.1, size=n)
dat = np.hstack((y2.reshape(n, 1), x2))
np.savetxt("data/reg_wide.txt", dat, fmt="%f", delimiter="\t")

PySpark 读取文件并进行一些简单操作：

In [ ]:
file1 = sc.textFile("data/reg_tall.txt")

# 打印矩阵行数
print(file1.count())

# 空行
print()

# 打印前5行，并将每行字符串截尾
text = file1.map(lambda x: x[:70] + "...").take(5)
print(*text, sep="\n")

In [ ]:
file2 = sc.textFile("data/reg_wide.txt")

# 打印矩阵行数
print(file2.count())

# 空行
print()

# 打印前5行，并将每行字符串截尾
text = file2.map(lambda x: x[:70] + "...").take(5)
print(*text, sep="\n")

### 2. $n\gg p$

回归系数估计值的显式解为 $\hat{\beta}=(X'X)^{-1}X'y$。当 $n\gg p$ 且 $p$ 不太大时，$X'X$ 为 $p\times p$ 矩阵，$X'y$ 为 $p\times 1$ 向量，均可放入内存。因此，此时问题的核心在于计算 $X'X$ 与 $X'y$。

首先进行分区映射：

In [ ]:
file_p10 = file1.repartition(10)
print(file_p10.getNumPartitions())

In [ ]:
# str => np.array
def str_to_vec(line):
    # 分割字符串
    str_vec = line.split("\t")
    # 将每一个元素从字符串变成数值型
    num_vec = map(lambda s: float(s), str_vec)
    # 创建 Numpy 向量
    return np.fromiter(num_vec, dtype=float)

# Iter[str] => Iter[matrix]
def part_to_mat(iterator):
    # Iter[str] => Iter[np.array]
    iter_arr = map(str_to_vec, iterator)

    # Iter[np.array] => list(np.array)
    dat = list(iter_arr)

    # list(np.array) => matrix
    if len(dat) < 1:  # Test zero iterator
        mat = np.array([])
    else:
        mat = np.vstack(dat)

    # matrix => Iter[matrix]
    yield mat

In [ ]:
dat = file_p10.mapPartitions(part_to_mat).filter(lambda x: x.shape[0] > 0)
print(dat.count())

In [ ]:
dat.first()

注意此时每个分区上的数据同时包含了因变量和自变量，在使用自变量时，要将第一列排除。计算 $X'X$：

In [ ]:
xtx = dat.map(lambda part: part[:, 1:].transpose().dot(part[:, 1:])).reduce(lambda x, y: x + y)
xtx

计算 $X'y$：

In [ ]:
xty = dat.map(lambda part: part[:, 1:].transpose().dot(part[:, 0])).reduce(lambda x, y: x + y)
xty

此时剩下的操作即为求解线性方程组。由于 $p$ 较小，故可以在内存中完成：

In [ ]:
bhat = np.linalg.solve(xtx, xty)
bhat

与真值进行对比：

In [ ]:
beta1

**思考题**：实际计算回归时，我们一般会加入截距项。此时应该如何修改程序，使其可以输出包含截距项的回归系数？

### 3. $n<p$

首先获取维度信息：

In [ ]:
n = file2.count()
n

In [ ]:
p = str_to_vec(file2.first()).shape[0] - 1
p

然后创建分区 RDD：

In [ ]:
dat = file2.repartition(10).mapPartitions(part_to_mat).filter(lambda x: x.shape[0] > 0)
print(dat.count())

当 $n<p$ 时，$X'X$ 不再可逆，因此最小二乘估计没有唯一解。此时我们可以采用岭回归的方法，其在最小二乘损失函数的基础上加入一个惩罚项 $\lambda \Vert\beta\Vert^2$。岭回归估计的显式解为 $\hat{\beta}_\lambda=(X'X+\lambda I)^{-1}X'y$，其中 $\lambda>0$ 是一个给定的正数。

但注意到 $X'X+\lambda I$ 是一个高维的矩阵，难以直接进行求解。因此我们采用共轭梯度法（参见 [lec7-cg.ipynb](lec7-cg.ipynb)）：

In [ ]:
def cg(Afn, b, x0, eps=1e-3, print_progress=False, **Afn_args):
    m = b.shape[0]
    # 初始解（注意此处应该复制x0，否则程序退出时会修改x0）
    x = np.copy(x0)
    # 初始残差向量
    r = b - Afn(x, **Afn_args)
    # 初始共轭梯度
    p = r

    for k in range(m):
        # 矩阵乘法
        Ap = Afn(p, **Afn_args)
        rr = r.dot(r)
        alpha = rr / p.dot(Ap)
        # 更新解
        x += alpha * p
        # 计算新残差向量
        rnew = r - alpha * Ap
        # 测试是否收敛
        norm = np.linalg.norm(rnew)
        if print_progress:
            print(f"Iter {k}, residual norm = {norm}")
        if norm < eps:
            break
        beta = rnew.dot(rnew) / rr
        # 更新共轭梯度
        p = rnew + beta * p
        # 更新残差向量
        r = rnew

    return x

先计算 $b=X'y$：

In [ ]:
b = dat.map(lambda part: part[:, 1:].transpose().dot(part[:, 0])).reduce(lambda x, y: x + y)
b

我们需要定义一个函数计算 $(X'X+\lambda I)v=X'Xv+\lambda v$，其中第一项可以分布式进行（参见笔记）。

In [ ]:
def xtxv(part, v):
    x = part[:, 1:]
    return x.transpose().dot(x.dot(v))

def ridge_prod(v, lam, rdd):
    first_term = rdd.map(lambda part: xtxv(part, v)).reduce(lambda x, y: x + y)
    second_term = lam * v
    return first_term + second_term

接下来调用 CG 函数，取 $\lambda=0.01 n$：

In [ ]:
lam = 0.01 * n
sol = cg(ridge_prod, b, x0=np.zeros(shape=p), eps=1e-3, print_progress=True, lam=lam, rdd=dat)

In [ ]:
sol[:30]

关闭 Spark 连接：

In [ ]:
sc.stop()